#### Setting environments

In [2]:
import torch, os, gc
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score,\
f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

In [6]:
# Set options
embed_ver = ["clstm", "esm2", "bert", "t5"]
comb_ver = ["emb_gen-clstm", "emb_gen-esm2", "emb_gen-bert", "emb_gen-t5"]

# model options
data_path = "../data/"
model_path = f"../model/emb_gen/"
result_path = f"../result/prd-emb_gen/"
os.makedirs(result_path, exist_ok=True)

col_str = ['file_id', 'organism', 'locus_tag', 'ess']

batch_size = 256

In [4]:
# Set data list for test dataset
ts_data = {
    "data1": ["C018"],  # "Escherichia coli K-12 BW25113"
    "data2": ["C016"],  # "Escherichia coli K-12 MG1655"
    "data3": ["O046"],  # "synthetic bacterium JCVI-Syn3A"
    "data4": ["C048"],  # Bacteroides thetaiotaomicron VPI-5482
    "data5": ["C050"]  # Salmonella enterica subsp. enterica serovar Typhimurium str. 14028S
}

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# Define function to record perfomance result
def record_perform(emb_ver, file_id, organ, y_real, y_conf, y_prd):
    y_real = y_real.cpu().numpy()
    y_conf = y_conf.cpu().numpy()
    y_prd = y_prd.cpu().numpy()
    
    if file_id != "O046":
        auc_roc = [roc_auc_score(y_real, y_conf)]
        auc_pr = [average_precision_score(y_real, y_conf)]
    else:
        auc_roc = None
        auc_pr = None
    
    tn, fp, fn, tp = confusion_matrix(y_real, y_prd).ravel()
    
    result = pd.DataFrame({
        "emb": [emb_ver],
        "file": [file_id],
        "organism": [organ],
        "tp": [tp],
        "fp": [fp],
        "tn": [tn],
        "fn": [fn],
        "mcc": [matthews_corrcoef(y_real, y_prd)],
        "acc": [accuracy_score(y_real, y_prd)],
        "f1": [f1_score(y_real, y_prd)],
        "prc": [precision_score(y_real, y_prd)],
        "rec": [recall_score(y_real, y_prd)],
        "npv": [precision_score(1 - y_real, 1 - y_prd)],
        "tnr": [recall_score(1 - y_real, 1 - y_prd)],
        "auc-roc": auc_roc,
        "auc-pr": auc_pr
    })

    return result


In [3]:
# Set model architecture
class Classifier(nn.Module):
    def __init__(self, input_size, num_layers, unit_decrease):
        super(Classifier, self).__init__()
        layers = [nn.BatchNorm1d(input_size), nn.Dropout(0.5)]
        in_dim = input_size
        out_dim = 1024
        for i in range(num_layers):            
            out_dim = max(2, out_dim // unit_decrease)
            layers.append(nn.Linear(in_dim, out_dim))
            self.initialize_weights(layers[-1])
            layers.append(nn.GELU())
            in_dim = out_dim
        layers.append(nn.Linear(out_dim, 1))
        self.cls_block = nn.Sequential(*layers)
        
    def initialize_weights(self, layer):
        nn.init.kaiming_normal_(layer.weight, mode='fan_in', nonlinearity='linear')
        if layer.bias is not None:
            nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        return self.cls_block(x)

#### Evaluate model

In [8]:
df_eval = pd.DataFrame()

for e_ver in embed_ver:
    ver = f'emb_gen-{e_ver}'
    print(f"\n>>>> {ver} <<<<")
    
    if ver in comb_ver:            
        # load dataset
        data = pd.read_csv(data_path + f"data-{ver}-ts.csv")
        display("Raw data:", data)

        #### Preprocess for test dataset ####
        col_num = [col for col in data.columns if col not in col_str]

        # get test datasets
        loc_ts = {}
        data_ts = {}
        org_ts = {}
        for ts_ver, ids in ts_data.items():
            # get test sample locations
            loc_ts[ts_ver] = data['file_id'].isin(ids)
            # get test samples
            data_ts[ts_ver] = data[loc_ts[ts_ver]]
            org = []
            # get test organism list
            for i in ids:
                organ = data_ts[ts_ver]['organism'][data_ts[ts_ver]['file_id'] == i].to_list()
                if len(organ) > 0:
                    org.append(organ[0])
            org_ts[ts_ver] = org

            print("Test dataset(" + ts_ver + "):", data_ts[ts_ver].shape)
        print("Test organism:", org_ts, len(org_ts))

        # split info.& inputs & labels of the test datasets
        info_ts = {}
        y_ts = {}
        test_loader = {}
        for ts_ver, df in data_ts.items():
            info_ts[ts_ver] = df[col_str]
            X_ts = torch.tensor(df[col_num].astype('float32').values)
            y_ts[ts_ver] = torch.tensor(df['ess'].astype('float32').values)
            print("Splited test dataset(" + ts_ver + "):", X_ts.shape, y_ts[ts_ver].shape)                    
            # generate dataloader by the test datasets
            dataset_ts = TensorDataset(X_ts, y_ts[ts_ver])
            test_loader[ts_ver] = DataLoader(dataset_ts, batch_size=batch_size, shuffle=False)

        # get the total test dataset
        loc_ts_all = [sum(loc) >= 1 for loc in zip(*loc_ts.values())]
        info_ts_all = data.loc[loc_ts_all, col_str]
        X_ts_all = torch.tensor(data.loc[loc_ts_all, col_num].astype('float32').values)
        y_ts_all = torch.tensor(data.loc[loc_ts_all, 'ess'].astype('float32').values)
        
        print("Splited test dataset(all):", X_ts_all.shape, y_ts_all.shape)

        # generate dataloader of total test dataset
        test_all_dataset = TensorDataset(X_ts_all, y_ts_all)
        test_all_loader = DataLoader(test_all_dataset, batch_size=batch_size, shuffle=False)
        

        #### Evaluate model ####
        # set model name
        model_name = f"{ver}-{fcnn_ver}"
        print(f"\n===== Test model: {model_name} ====")
        # generate model instance
        model = Classifier(
            input_size=X_ts_all.shape[-1],
            num_layers=3,
            unit_decrease=2
        ).to(device)

        # load model weight
        model.load_state_dict(torch.load(model_path + model_name + ".pt", map_location=device))
        model.eval()


        ## model evaluations by test dataset ##
        for ts_ver, ids in ts_data.items():
            all_preds = []
            all_labels = []
            with torch.no_grad():
                for X_batch, y_batch in test_loader[ts_ver]:
                    X_batch = X_batch.to(device)
                    y_batch = y_batch.to(device)
                    preds = model(X_batch).squeeze()
                    all_preds.append(preds.cpu())
                    all_labels.append(y_batch.cpu())
            
            # concatenate results to one tensor
            all_preds = torch.cat(all_preds)
            all_labels = torch.cat(all_labels)

            # convert logits to confidences & classes
            prd_conf = torch.sigmoid(all_preds)
            prd_cls = (prd_conf >= 0.5).int()
            # performances by testset
            perform = record_perform(
                emb_ver=e_ver,
                file_id="+".join(ids),
                organ="+".join(org_ts[ts_ver]),
                y_real=y_ts[ts_ver],
                y_conf=prd_conf,
                y_prd=prd_cls,
            )
            display(perform)
            df_eval = pd.concat([df_eval, perform], ignore_index=True)
        
        
        ## model evaluation on the total test dataset ##
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for X_batch, y_batch in test_all_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                preds = model(X_batch).squeeze()
                all_preds.append(preds.cpu())
                all_labels.append(y_batch.cpu())
        
        # concatenate results to one tensor
        all_preds = torch.cat(all_preds)
        all_labels = torch.cat(all_labels)

        # convert logits to confidences & classes
        prd_conf = torch.sigmoid(all_preds)
        prd_cls = (prd_conf >= 0.5).int()

        # performances on total testset
        perform = record_perform(
            emb_ver=e_ver,
            file_id="+".join([i for ids in ts_data.values() for i in ids]),
            organ="+".join([org for orgs in org_ts.values() for org in orgs]),
            y_real=y_ts_all,
            y_conf=prd_conf,
            y_prd=prd_cls
        )
        display(perform)
        df_eval = pd.concat([df_eval, perform], ignore_index=True)

        # concatenate the protein info. & predicted confidences
        df_prd = pd.DataFrame(prd_conf, columns=["conf"], index=info_ts_all.index)
        df_prd = pd.concat([info_ts_all, df_prd], axis=1)

        # save the model prediction result
        df_prd.to_csv(result_path + f"prd-{model_name}.csv", index=False)

        gc.collect()

# save the model perfomance result
display("Model performance:", df_eval)
df_eval.to_csv(f"../result/eval-emb_gen.csv", index=False)


>>>> emb_gen-clstm <<<<


'Raw data:'

,file_id,organism,locus_tag,ess,0,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0001,0,-1.271414,0.000099,0.076290,-0.000540,-0.002149,-0.002770,...,-0.000128,0.004823,0.000413,0.127032,0.006706,0.013683,-0.020618,-0.049667,0.108772,-0.003629
1,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0002,0,0.080849,0.000191,0.066948,-0.001360,-0.001139,-0.002485,...,-0.000043,0.001776,0.001967,0.126776,0.007055,0.008662,0.113034,-0.035381,0.105291,0.002070
2,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0003,0,0.049742,0.000037,0.063013,-0.003911,-0.002671,-0.000340,...,-0.000037,0.001271,0.002466,0.126992,0.007107,0.007993,0.119393,-0.034128,0.103778,0.002723
3,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0004,0,0.438240,0.000130,0.061992,-0.002766,-0.001883,-0.001800,...,-0.000041,0.001173,0.001617,0.127110,0.007114,0.008159,0.119812,-0.033128,0.105875,0.002058
4,C050,Salmonella enterica subsp. enterica serovar Ty...,STM14_0005,0,0.647014,0.000091,0.058150,-0.003637,-0.002623,-0.002049,...,-0.000028,0.000526,0.001174,0.127482,0.007209,0.007493,0.129736,-0.029263,0.105677,0.004036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19378,O046,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0913,1,0.244533,0.000047,0.060919,-0.003986,-0.002723,-0.000765,...,-0.000052,0.001888,0.002917,0.127124,0.007069,0.007943,0.102165,-0.032948,0.106551,0.001652
19379,O046,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0918,1,0.446268,0.000114,0.060191,-0.002962,-0.002494,-0.002787,...,-0.000028,0.001444,0.000616,0.126796,0.007066,0.008400,0.101383,-0.035993,0.105889,0.002418
19380,O046,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0930,1,-1.313249,0.000093,0.077817,-0.000610,-0.002145,-0.002522,...,-0.000083,0.002701,0.002104,0.127490,0.007041,0.009210,0.034684,-0.041377,0.106090,-0.001276
19381,O046,synthetic bacterium JCVI-Syn3A,JCVISYN3A_0931,1,0.339057,0.000057,0.060354,-0.003875,-0.002801,-0.001344,...,-0.000051,0.000935,0.001868,0.126976,0.007136,0.007776,0.122922,-0.030382,0.105050,0.003196


Test dataset(data1): (4313, 1028)
Test dataset(data2): (4313, 1028)
Test dataset(data3): (458, 1028)
Test dataset(data4): (4825, 1028)
Test dataset(data5): (5474, 1028)
Test organism: {'data1': ['Escherichia coli K-12 BW25113'], 'data2': ['Escherichia coli K-12 MG1655'], 'data3': ['synthetic bacterium JCVI-Syn3A'], 'data4': ['Bacteroides thetaiotaomicron VPI-5482'], 'data5': ['Salmonella enterica subsp. enterica serovar Typhimurium str. 14028S']} 5
Splited test dataset(data1): torch.Size([4313, 1024]) torch.Size([4313])
Splited test dataset(data2): torch.Size([4313, 1024]) torch.Size([4313])
Splited test dataset(data3): torch.Size([458, 1024]) torch.Size([458])
Splited test dataset(data4): torch.Size([4825, 1024]) torch.Size([4825])
Splited test dataset(data5): torch.Size([5474, 1024]) torch.Size([5474])
Splited test dataset(all): torch.Size([19383, 1024]) torch.Size([19383])

===== Test model: emb_gen-clstm-3_2 ====


/tmp/ipykernel_7278/1828744696.py:73: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path + model_name + ".pt", map_location=device))


...

,emb,fcnn,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,t5,3_2,data4,C048,Bacteroides thetaiotaomicron VPI-5482,257,529,3971,68,0.456933,0.876269,0.462646,0.326972,0.790769,0.983164,0.882444,0.918049,0.616563


,emb,fcnn,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,t5,3_2,data5,C050,Salmonella enterica subsp. enterica serovar Ty...,36,651,4710,77,0.084616,0.867008,0.09,0.052402,0.318584,0.983915,0.878567,0.781696,0.047115


,emb,fcnn,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,t5,3_2,test_all,C018+C016+O046+C048+C050,Escherichia coli K-12 BW25113+Escherichia coli...,1150,1921,15978,334,0.486138,0.883661,0.50494,0.374471,0.774933,0.979524,0.892676,0.900482,0.525443


'Model performance:'

,emb,fcnn,testset,file,organism,tp,fp,tn,fn,mcc,acc,f1,prc,rec,npv,tnr,auc-roc,auc-pr
0,clstm,3_2,data1,C018,Escherichia coli K-12 BW25113,233,229,3786,65,0.594411,0.931834,0.613158,0.504329,0.781879,0.983121,0.942964,0.910965,0.586572
1,clstm,3_2,data2,C016,Escherichia coli K-12 MG1655,241,240,3783,49,0.613699,0.932993,0.625162,0.501040,0.831034,0.987213,0.940343,0.947629,0.632228
2,clstm,3_2,data3,O046,synthetic bacterium JCVI-Syn3A,112,0,0,346,0.000000,0.244541,0.392982,1.000000,0.244541,0.000000,0.000000,NaN,NaN
3,clstm,3_2,data4,C048,Bacteroides thetaiotaomicron VPI-5482,282,107,4393,43,0.776919,0.968912,0.789916,0.724936,0.867692,0.990307,0.976222,0.975421,0.818815
4,clstm,3_2,data5,C050,Salmonella enterica subsp. enterica serovar Ty...,78,493,4868,35,0.278314,0.903544,0.228070,0.136602,0.690265,0.992862,0.908040,0.891886,0.107095
5,clstm,3_2,test_all,C018+C016+O046+C048+C050,Escherichia coli K-12 BW25113+Escherichia coli...,946,1069,16830,538,0.503330,0.917092,0.540726,0.469479,0.637466,0.969023,0.940276,0.883538,0.461305
6,esm2,3_2,data1,C018,Escherichia coli K-12 BW25113,269,519,3496,29,0.507602,0.872942,0.495396,0.341371,0.902685,0.991773,0.870735,0.942126,0.701041
7,esm2,3_2,data2,C016,Escherichia coli K-12 MG1655,280,517,3506,10,0.540069,0.877811,0.515179,0.351317,0.965517,0.997156,0.871489,0.975500,0.784774
8,esm2,3_2,data3,O046,synthetic bacterium JCVI-Syn3A,259,0,0,199,0.000000,0.565502,0.722455,1.000000,0.565502,0.000000,0.000000,NaN,NaN
9,esm2,3_2,data4,C048,Bacteroides thetaiotaomicron VPI-5482,292,622,3878,33,0.486274,0.864249,0.471348,0.319475,0.898462,0.991562,0.861778,0.937444,0.613368
